## Investments Assignment 7

#### Anita Mezzetti, Hien Le, Francesco Maizza

In [66]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import random as random
import matplotlib.pyplot as plt
from datetime import datetime
import statsmodels.api as sm
import wrds
import warnings
from scipy import stats
warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
db = wrds.Connection(wrds_username = 'anita97') 

Loading library list...
Done


## a)

In [95]:
#import stocks
'''s = db.raw_sql("select a.permno, a.date, b.exchcd, \
                  a.ret, b.shrcd, a.shrout, a.prc \
                  from crsp.msf as a \
                  join crsp.msenames as b \
                  on a.permno = b.permno and b.namedt<=a.date \
                  and a.date<=b.nameendt \
                  where b.shrcd between (10) and (11) and \
                  a.date >='2000-01-01' and a.date<='2019-12-31'\
                  and b.exchcd between 1 and 2")'''

#s.to_csv('as7.csv')
stocks = pd.read_csv('as7.csv', index_col=0) # read the file

For each value we want one value for each month. Firstly we trasform data column in month-year(format). Then, for each stock we chack that there is only one value for each month:

In [96]:
stocks.date = pd.DatetimeIndex(stocks.date) # data format
stocks.date = stocks.date.dt.to_period('M') # data year-month
stocks = stocks.drop_duplicates(['permno','date']) # delete duplicates
stocks.sort_values(by=['permno','date'])
stocks = stocks.set_index('permno') # set permno as index

In [97]:
# We set also date as index for having a nice output
stocks_output = stocks.reset_index()
stocks_output = stocks_output.set_index(['permno','date'])
stocks_output.head()

exchcd       ret  shrout  shrcd      prc
permno  date                                             
10001.0 2009-12     2.0  0.162621  4361.0   11.0  10.3000
        2010-01     2.0 -0.018932  4361.0   11.0  10.0600
        2010-02     2.0 -0.000656  4361.0   11.0  10.0084
        2010-03     2.0  0.020643  4361.0   11.0  10.1700
        2010-04     2.0  0.124385  6070.0   11.0  11.3900

We have to delete all the stocks which have less than 240 observations. In order to do that, we can group all the stocks on permno (which identify one stock) and then count how many observations we have for each stock:

In [98]:
count = []
count = stocks.groupby('permno').count()
# now in each column we have the number of observations. We can rename one of them
count = count.rename(columns={"date":"number of observations"})
count = count[['number of observations']]
print(f"We have {count.shape[0]} different stocks. Each has a certain number of observations:")
count.head()

We have 4118 different stocks. Each has a certain number of observations:


,number of observations
permno,
10001.0,92
10028.0,140
10042.0,98
10051.0,233
10066.0,28


Now we delete  stocks with less than 240 observations:

Note: This is equal to have only stocks with preciselt 240 observations. Because we want that they are traded EVERY mounth for 20 years: 

20 years*12 mounths = 240 observation. 

In [99]:
decision = count[['number of observations']]!=240 # delete if true
decision = decision.dropna() # drop NaN
decision.head()

,number of observations
permno,
10001.0,True
10028.0,True
10042.0,True
10051.0,True
10066.0,True


In [100]:
decision_true = decision[decision['number of observations']] # keep only true
print(f"We delete {decision_true.shape[0]} stocks.")
stocks_del = stocks.drop(index=decision_true.index) # drop the stocks that have < 240 observations
stocks_del.head()

We delete 3479 stocks.


,date,exchcd,ret,shrout,shrcd,prc
permno,,,,,,
10145.0,2000-01,1.0,-0.167931,789233.0,11.0,48.000000
10145.0,2001-07,1.0,0.059088,809281.0,11.0,36.869999
10145.0,2001-08,1.0,0.015663,811616.0,11.0,37.259998
10145.0,2001-09,1.0,-0.291465,811616.0,11.0,26.400000
10145.0,2001-10,1.0,0.119318,811619.0,11.0,29.549999


We should have 639 stocks:

In [101]:
# we have deleted decision_true
# we are left with the rest of decision
num_deleted = int(decision_true.count())
num_left = int(decision.count()-decision_true.count())
print(f"We have deleted {num_deleted} and we are left with {num_left} stocks")

We have deleted 3479 and we are left with 639 stocks


Delete 3 NaN stocks: fisrt we delete null returns. This operation doesn not remove all the stock obersavations, but only the null ones. In order to cancel all the observations of stocks with NaN values, we re-check that stocks actually have 240 obersations.

In [102]:
is_na = stocks_del.ret.isna()
num_before = len(stocks_del.index)
stocks_del = stocks_del[~is_na] # delete null returns
num_then = len(stocks_del.index)
if any(is_na) == True:
    print(f"{num_before-num_then} returns are NaN")

3 returns are NaN


We are to cancel not only the row with nan return, but allthe stock, which now it has not 240 observations.

In [103]:
count = []
count = stocks_del.groupby('permno').count()

# now in each column we have the number of observations. 
#We can rename one of them
count = count.rename(columns={"date":"number of observations"})
count = count[['number of observations']]

decision = count[['number of observations']]!=240 # delete if true
#decision = decision.dropna() # drop NaN

In [104]:
decision_true = decision[decision['number of observations']] # keep only true
print(f"We delete {decision_true.shape[0]} stocks.")
stocks_del = stocks_del.drop(index=decision_true.index) # drop the stocks that have < 240 observations
stocks_del.head()

We delete 3 stocks.


,date,exchcd,ret,shrout,shrcd,prc
permno,,,,,,
10145.0,2000-01,1.0,-0.167931,789233.0,11.0,48.000000
10145.0,2001-07,1.0,0.059088,809281.0,11.0,36.869999
10145.0,2001-08,1.0,0.015663,811616.0,11.0,37.259998
10145.0,2001-09,1.0,-0.291465,811616.0,11.0,26.400000
10145.0,2001-10,1.0,0.119318,811619.0,11.0,29.549999


In [105]:
# we have deleted decision_true
# we are left with the rest of decision
num_deleted = int(decision_true.count())
num_left = int(decision.count()-decision_true.count())
print(f"We have deleted {num_deleted} and we are left with {num_left} stocks")

We have deleted 3 and we are left with 636 stocks


In [106]:
is_na = stocks_del.ret.isna()
num_before = len(stocks_del.index)
stocks_del = stocks_del[~is_na] # delete null returns
num_then = len(stocks_del.index)
if any(is_na) == True:
    print(f"{num_before-num_then} returns are NaN")
else:
    print("Now we do not have NaN returns")

Now we do not have NaN returns


Now, we can add the risk free rate and market returns. 

In [107]:
# import risk free rate
risk_free = db.raw_sql("select mcaldt as date, tmytm as rf from crsp.tfz_mth_rf where kytreasnox = 2000001 and mcaldt>='2000-01-01' and mcaldt<='2019-12-31'")
risk_free_m = np.exp(np.divide(risk_free.rf, 1200)) -1
rf = pd.DataFrame([risk_free.date, risk_free_m], index=['date', 'rf']).T

# import market
market = db.raw_sql("select date, vwretd from crsp.msi where date>='2000-01-01' and date<='2019-12-31'")
market_m = pd.DataFrame([market.date, market.vwretd.values], index=['date', 'mkt_ret']).T
rm = market_m.set_index('date')

In [108]:
rf.date = pd.DatetimeIndex(rf.date) # data format
rf.date = rf.date.dt.to_period('M') # data year-month
rf = rf.set_index('date')
rf.head()

,rf
date,
2000-01,0.00449835
2000-02,0.00443834
2000-03,0.00489188
2000-04,0.00440126
2000-05,0.00390314


In [109]:
rm.index = pd.DatetimeIndex(rm.index) # data format
rm.index = rm.index.to_period('M') # data year-month
rm.head()

,mkt_ret
date,
2000-01,-0.0396243
2000-02,0.0317784
2000-03,0.0535007
2000-04,-0.059527
2000-05,-0.0388696


In [110]:
save = stocks_del

In [250]:
stocks_del = save

## b)

Caculate market capitalization, making sure that we have absolute values of share prices:

In [251]:
stocks_del['prc'] = stocks_del['prc'].abs() # absolute value of the share price
stocks_del['mrk_cap'] = stocks_del['prc'] * stocks_del['shrout']
stocks_del.head()

,date,exchcd,ret,shrout,shrcd,prc,mrk_cap
permno,,,,,,,
10145.0,2000-01,1.0,-0.167931,789233.0,11.0,48.000000,3.788318e+07
10145.0,2001-07,1.0,0.059088,809281.0,11.0,36.869999,2.983819e+07
10145.0,2001-08,1.0,0.015663,811616.0,11.0,37.259998,3.024081e+07
10145.0,2001-09,1.0,-0.291465,811616.0,11.0,26.400000,2.142666e+07
10145.0,2001-10,1.0,0.119318,811619.0,11.0,29.549999,2.398334e+07


We calculate the beta

In [252]:
# list of right permnos (no duplicates):
list_permno = list(stocks_del.loc[~stocks_del.index.duplicated(keep='first')].index)

market_beta = [] # market beta 

#cycle for each stock (639 elemets)
for i in list_permno: # (list_permno): 
    current_stock = stocks_del.loc[i] # long 240 because we have 240 observations for each stock
    ret_cur_stock = current_stock.ret.values.astype(float) # 240 returns for each stock
    rm_cur_stock = rm.loc[current_stock.date].mkt_ret.astype(float)
    covariance = np.cov(ret_cur_stock, rm_cur_stock) # Cov(R,Rm)
    var_market = covariance[1,1] # var(Rm)
    market_beta.append(covariance[0,1]/var_market) # add new beta
    
# Create a DataFrame where for each stock we have its beta
data = pd.DataFrame(market_beta, columns={'beta'}, index=list_permno) # for each stock we have its beta
data = data.sort_index()

print(type(stocks_del))

# We add to the stocks dataframe our beta, creating a new column:
stocks_del = stocks_del.merge(data, left_index=True, right_index=True)
#stocks_del = stocks_del.drop(stocks_del.loc[stocks_del.beta.isnull()].index)#.set_index(['permno','date']) # re-index
stocks_del.head()

<class 'pandas.core.frame.DataFrame'>


,date,exchcd,ret,shrout,shrcd,prc,mrk_cap,beta
10145.0,2000-01,1.0,-0.167931,789233.0,11.0,48.000000,3.788318e+07,1.20019
10145.0,2001-07,1.0,0.059088,809281.0,11.0,36.869999,2.983819e+07,1.20019
10145.0,2001-08,1.0,0.015663,811616.0,11.0,37.259998,3.024081e+07,1.20019
10145.0,2001-09,1.0,-0.291465,811616.0,11.0,26.400000,2.142666e+07,1.20019
10145.0,2001-10,1.0,0.119318,811619.0,11.0,29.549999,2.398334e+07,1.20019


Form 10 groups of stocks based on the market capitalization of 31 dec 2019, with first decile being group 1 on so on:

In [253]:
# list of right permnos (no duplicates):
stocks_del = stocks_del.sort_index(axis = 0) 
list_permno = list(stocks_del.loc[~stocks_del.index.duplicated(keep='first')].index)

# consider market capitalization on dec 2019
stocks_dec2019 = stocks_del[stocks_del.date.astype(str).values=='2019-12']
stocks_dec2019 = stocks_dec2019.sort_index(axis = 0) 
stocks_dec2019.head()

,date,exchcd,ret,shrout,shrcd,prc,mrk_cap,beta
10145.0,2019-12,1.0,-0.008681,714533.0,11.0,177.000000,1.264723e+08,1.200190
10294.0,2019-12,1.0,0.006728,1535.0,11.0,20.200001,3.100700e+04,0.758752
10308.0,2019-12,2.0,0.054755,14940.0,11.0,91.500000,1.367010e+06,0.443625
10516.0,2019-12,1.0,0.079665,556686.0,11.0,46.349998,2.580240e+07,0.497710
10517.0,2019-12,1.0,-0.021404,67152.0,11.0,57.110001,3.835051e+06,0.642811


In [254]:
# creation on 10 groups
a = pd.qcut(stocks_dec2019.mrk_cap,10)
decile = pd.DataFrame(pd.qcut(stocks_dec2019.mrk_cap,10))
decile = decile.rename(columns={"mrk_cap":"decile"})
decile.head()

,decile
10145.0,"(54953204.591, 437225989.856]"
10294.0,"(2640.959, 221410.898]"
10308.0,"(848916.265, 1570788.675]"
10516.0,"(23590062.289, 54953204.591]"
10517.0,"(2728756.376, 4603126.817]"


In [255]:
inters = list(dict.fromkeys(a))
#inters.sort()
inters = [x for x in inters if str(x) != 'nan'] # drop NoN. We always put a control just to be sure
print(f"We have {len(inters)} groups of stcoks.")

We have 10 groups of stcoks.


In [256]:
# for each portfolio we have the decile to which it belongs
group_decile = decile.sort_values(by="decile") # sort for decile
group_decile.reset_index(inplace=True)
group_decile = group_decile.dropna() # drop NaN
group_decile = group_decile.groupby("decile")

In [257]:
# compute the size-wheighted returns for a given month 
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return np.nan

**EQUALLY WEIGHTED**

In [267]:
month_list = stocks_del.date.drop_duplicates(keep='first').values
stocks_del = stocks_del.sort_values('date') #sort for date
ew_ret = []

for month in month_list: # for each month 
    stocks_del_month = stocks_del[stocks_del.date.values==month]
    
    ew_ret_month = [] # returns of equally weighted portfolios
    for i in inters: # each decile
        stock_interval = group_decile.get_group(i)["index"]
        temp = stocks_del_month.loc[stock_interval] 
        #group_date = temp.groupby('date').mean()
        ew_ret_month.append(temp.ret.mean()) # equal weighted => mean
        

    ew_ret.append(ew_ret_month)

In [303]:
ew_ret[0:20]

[[-0.04503637546804384,
  0.05014570678395103,
  -0.050526173873102306,
  -0.04881972574170615,
  -0.04556045373753895,
  -0.057254742496297695,
  -0.010908959535299793,
  -0.05112946718872989,
  -0.04494867054927455,
  -0.05209665686365154],
 [-0.08797004241750986,
  0.08340600237715982,
  0.01052988952557954,
  -0.06958276130968616,
  -0.01442203855526355,
  -0.03083706038160021,
  0.03245743512889023,
  -0.057213702462831834,
  0.0014417665615837783,
  -0.05650094339978836],
 [0.12939912930596623,
  -0.0006115714313636366,
  0.07157198093565444,
  0.11019845771241105,
  0.10479464812115544,
  0.13405541903193807,
  0.03728747021523303,
  0.13783316466287943,
  0.05838366066745942,
  0.12982176130800144],
 [0.0397472745498817,
  -0.025198833100148463,
  -0.0018100975864818194,
  0.04639607484615036,
  0.022141934013075456,
  0.03928736781017503,
  -0.01731916984863346,
  0.016280367119179552,
  -0.0018071712620439714,
  0.011806581945468997],
 [0.025852004524494987,
  -0.043203438959

now we have an array where for each month we have 10 ew returns. we want the excess return on rf:

In [270]:
month_exc_ret_ew = []

for i in range(len(month_list)): # for each month
    risk_free = rf.rf[i] # rf of that month
    risk_free_list = [risk_free]*10 
    month_exc_ret_ew.append(np.asarray(ew_ret[i])-risk_free)

In month_exc_ret we have 10 excess returns for each month. Now we want I excess return for each decile, then we do the mean over all months:

**VALUE WEIGHTED**

In [272]:
# LAGGED market capitalization

stocks_del['lagged_mrk_cap'] =  stocks_del.groupby(stocks_del.index)['mrk_cap'].shift(1) # lagged market cap
print("we can see that lagged mark cap for each stock is NaN for the first month")
stocks_del.head()

we can see that lagged mark cap for each stock is NaN for the first month


,date,exchcd,ret,shrout,shrcd,prc,mrk_cap,beta,lagged_mrk_cap_formonth,lagged_mrk_cap
10145.0,2000-01,1.0,-0.167931,789233.0,11.0,48.0000,3.788318e+07,1.200190,NaN,NaN
80415.0,2000-01,1.0,-0.168590,19048.0,11.0,32.3750,6.166790e+05,1.060152,NaN,NaN
21055.0,2000-01,1.0,-0.326923,22002.0,11.0,8.7500,1.925175e+05,1.280285,NaN,NaN
78950.0,2000-01,1.0,0.072927,9736.0,11.0,10.9375,1.064875e+05,0.904525,NaN,NaN
80303.0,2000-01,1.0,-0.456954,103057.0,11.0,10.1250,1.043452e+06,2.427263,NaN,NaN


In [295]:
stocks_del.lagged_mrk_cap = stocks_del.lagged_mrk_cap.fillna(0) # fill nan with zero (zero weight)
month_list = stocks_del.date.drop_duplicates(keep='first').values
stocks_del = stocks_del.sort_values('date') #sort for date
vw_ret = []

for month in month_list: # for each month 
    stocks_del_month = stocks_del[stocks_del.date.values==month]
    
    vw_ret_month = [] # returns of equally weighted portfolios
    for i in inters: # each decile
        
        stock_interval = group_decile.get_group(i)["index"]
        temp = stocks_del_month.loc[stock_interval] # stocks for that month for that decile 
        
        # calculate weights
        weight_sum = temp.lagged_mrk_cap.sum() # sum of mark_cap
        weights = temp.lagged_mrk_cap / weight_sum #weights
        
        # calulate value weighted ret
        ret = (temp.ret*weights).sum()
        
        vw_ret_month.append(vw_ret)
        

    vw_ret.append(ew_ret_month)

In [304]:
vw_ret[0:20]

[[0.026050946693430888,
  0.04495055804613911,
  0.06608705632647291,
  0.03183513583748467,
  0.03461249389395577,
  0.03278840009670603,
  0.06548393547927842,
  0.024895685248338864,
  0.054230784586252405,
  0.031417000050991885],
 [0.026050946693430888,
  0.04495055804613911,
  0.06608705632647291,
  0.03183513583748467,
  0.03461249389395577,
  0.03278840009670603,
  0.06548393547927842,
  0.024895685248338864,
  0.054230784586252405,
  0.031417000050991885],
 [0.026050946693430888,
  0.04495055804613911,
  0.06608705632647291,
  0.03183513583748467,
  0.03461249389395577,
  0.03278840009670603,
  0.06548393547927842,
  0.024895685248338864,
  0.054230784586252405,
  0.031417000050991885],
 [0.026050946693430888,
  0.04495055804613911,
  0.06608705632647291,
  0.03183513583748467,
  0.03461249389395577,
  0.03278840009670603,
  0.06548393547927842,
  0.024895685248338864,
  0.054230784586252405,
  0.031417000050991885],
 [0.026050946693430888,
  0.04495055804613911,
  0.066087056

now we have an array where for each month we have 10 ew returns. we want the excess return on rf:

In [299]:
month_exc_ret_vw = []

for i in range(len(month_list)): # for each month
    risk_free = rf.rf[i] # rf of that month
    risk_free_list = [risk_free]*10 
    month_exc_ret_vw.append(np.asarray(vw_ret[i])-risk_free)

In [302]:
month_exc_ret_vw[0:20]

[array([0.0215526 , 0.04045221, 0.06158871, 0.02733679, 0.03011415,
        0.02829005, 0.06098559, 0.02039734, 0.04973244, 0.02691865]),
 array([0.0216126 , 0.04051221, 0.06164871, 0.02739679, 0.03017415,
        0.02835006, 0.06104559, 0.02045734, 0.04979244, 0.02697866]),
 array([0.02115907, 0.04005868, 0.06119518, 0.02694326, 0.02972062,
        0.02789652, 0.06059206, 0.02000381, 0.04933891, 0.02652512]),
 array([0.02164968, 0.04054929, 0.06168579, 0.02743387, 0.03021123,
        0.02838714, 0.06108267, 0.02049442, 0.04982952, 0.02701574]),
 array([0.02214781, 0.04104742, 0.06218392, 0.027932  , 0.03070936,
        0.02888526, 0.0615808 , 0.02099255, 0.05032765, 0.02751386]),
 array([0.02134897, 0.04024859, 0.06138508, 0.02713316, 0.02991052,
        0.02808643, 0.06078196, 0.02019371, 0.04952881, 0.02671503]),
 array([0.02114318, 0.0400428 , 0.06117929, 0.02692737, 0.02970473,
        0.02788064, 0.06057617, 0.01998792, 0.04932302, 0.02650924]),
 array([0.02084265, 0.03974226, 0.